In [1]:
import pandas as pd
import datetime
import os
import pytz
print(pd.__version__)
utc = pytz.utc

datDir = "./withings/raw_sleep-monitor_sleep-state.csv"



with open (datDir, "r") as myfile:
    dataSTRs=myfile.readlines()
dataSTRs = dataSTRs[1:]#remove the title bar

outputFileName = "withingsBedSleepStages.parquet.gzip"
outputTimezone = -7

dataFileColumns = ["timestamp", "stage", "endTimestamp"]


#get list of files in current directroy
files = [f for f in os.listdir("./") if os.path.isfile(f)]

print(files)
print(outputFileName in files)
#if the output file isn't already made, make it
if not (outputFileName in files):
    print("making new df")
    emptydf = pd.DataFrame(columns = dataFileColumns)
    emptydf.set_index('timestamp')
    print(emptydf)
        
    #save as a parquet file
    emptydf.to_parquet(outputFileName, compression='gzip') 


df = pd.read_parquet(outputFileName)

print(df.head(100))

1.2.4
['importHaloSleep.ipynb', 'importwithings.ipynb']
False
making new df
Empty DataFrame
Columns: [timestamp, stage, endTimestamp]
Index: []
Empty DataFrame
Columns: [timestamp, stage, endTimestamp]
Index: []


In [2]:
#ok I'm just going to assume that the gps corrdinates are necessary metadata to get the local time offset
#i'm just going to manually get all of the values from the data exports in UTC time and then calculate the local offset later

# timezone tests
testdt = datetime.datetime(2002, 10, 27, 6, 0, 0)
print(testdt)
print(testdt.tzinfo)
testdt = utc.localize(testdt)
print(testdt)
print(testdt.tzinfo)

2002-10-27 06:00:00
None
2002-10-27 06:00:00+00:00
UTC


In [3]:
#readlines
#the problem where if there is only one element in the list there are no quotes used so the formatting is slightly diffrent 

for r in dataSTRs:
    datList = r.split('[')
    toadd = []

    time = pd.to_datetime(datList[0].split("+")[0], format='%Y-%m-%dT%H:%M:%S') 
    tzoffset = datetime.timedelta(hours=(-1 * int(datList[0].split("+")[1][1])))#it's stored in french time so the offset should always be +
    startTime = utc.localize(time + tzoffset)
    stageStartTime = startTime
    toadd.append(stageStartTime)
    #print(stageStartTime)
    #print(tzoffset)

    #first i want to check if the datList[2]s length is == 2 and if so parse it diffrently, write the toadd and continue
    if len(datList[2].strip()) == 2:
        toadd.append(int(datList[2][0]))
        toadd.append(stageStartTime + datetime.timedelta(minutes=1))
        df.loc[stageStartTime] = toadd
    else:
        stagelist = datList[2].strip()[:-2].split(',') #remove newline, clsose bracket and close quote, and split on commas
        #print(stagelist)
        
        #get the first stage in the row and add it to tooadd and set it to lastStage
        toadd.append(int(stagelist[0]))
        for i in range(1,len(stagelist)):
            if stagelist[i] != stagelist[i-1]:
                #save the current time as the end of the last stage
                toadd.append(startTime + datetime.timedelta(minutes=i))
                #add to the df
                df.loc[stageStartTime] = toadd
                #reset toadd
                toadd = []
                #set the current time to the start of the next one
                stageStartTime = startTime + datetime.timedelta(minutes=i)
                toadd.append(stageStartTime)
                #set the lable of the next stage to the value of the current index
                toadd.append(int(stagelist[i]))
        #set the last end time to the last toadd
        toadd.append(startTime + datetime.timedelta(minutes=len(stagelist)))
        #add it to the df
        df.loc[stageStartTime] = toadd





#for each line
    #seperate out the initial timestamp
    #get the list of sleep stages
    #for every stage in that list
        #increment offset
        #see if that is a number is new
        #if so end and write the current sleep stage
        #start the new stage

In [4]:
print(df.head(20))

                                          timestamp stage  \
index                                                       
2021-01-15 03:09:00+00:00 2021-01-15 03:09:00+00:00     0   
2021-01-15 03:21:00+00:00 2021-01-15 03:21:00+00:00     1   
2021-01-15 03:36:00+00:00 2021-01-15 03:36:00+00:00     0   
2021-01-15 07:30:00+00:00 2021-01-15 07:30:00+00:00     0   
2021-01-15 08:52:00+00:00 2021-01-15 08:52:00+00:00     1   
2021-01-15 09:01:00+00:00 2021-01-15 09:01:00+00:00     2   
2021-01-15 09:20:00+00:00 2021-01-15 09:20:00+00:00     1   
2021-01-15 09:35:00+00:00 2021-01-15 09:35:00+00:00     3   
2021-01-15 09:45:00+00:00 2021-01-15 09:45:00+00:00     1   
2021-01-15 10:30:00+00:00 2021-01-15 10:30:00+00:00     1   
2021-01-15 11:14:00+00:00 2021-01-15 11:14:00+00:00     2   
2021-01-15 11:28:00+00:00 2021-01-15 11:28:00+00:00     3   
2021-01-15 12:26:00+00:00 2021-01-15 12:26:00+00:00     1   
2021-01-15 13:30:00+00:00 2021-01-15 13:30:00+00:00     1   
2021-01-15 13:33:00+00:0

In [5]:
df.to_parquet(outputFileName, compression='gzip') 